In [45]:
import math
import geopy.distance
from dis import dis
import math
import geopandas as gpd
import numpy
from shapely import wkt
from shapely import wkb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim

%matplotlib inline

train_data = pd.read_csv('../data/stores_train.csv')
busstops = pd.read_csv('../data/busstops_norway.csv')
grunnkrets_age = pd.read_csv('../data/grunnkrets_age_distribution.csv')
grunnkrets_households = pd.read_csv('../data/grunnkrets_households_num_persons.csv')
grunnkrets_income = pd.read_csv('../data/grunnkrets_income_households.csv')
grunnkrets_stripped = pd.read_csv('../data/grunnkrets_norway_stripped.csv')
plaace_hierarchy = pd.read_csv('../data/plaace_hierarchy.csv')

In [46]:
train_data.head()

,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name,revenue
0,983540538-974187930-44774,2016,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,Hamburger restaurants,6020303,BRAGERNES TORG 13,59.743104,10.204928,MCDONALDS,Magasinet Drammen,17.998
1,987074191-973117734-44755,2016,MCDONALD'S KLINGENBERGGATA,1.1.1.0,Hamburger restaurants,3010306,NaN,59.913759,10.734031,MCDONALDS,NaN,23.828
2,984890265-981157303-64491,2016,BURGER KING HØNEFOSS,1.1.1.0,Hamburger restaurants,6050102,KONG RINGS GATE 1,60.164751,10.254656,BURGER KING,Kuben Hønefoss,16.099
3,914057442-992924179-126912,2016,BURGER KING GLASSHUSPASSASJEN,1.1.1.0,Hamburger restaurants,18040102,STORGATA 12,67.283669,14.379796,BURGER KING,Glasshuspassasjen,9.296
4,913018583-913063538-668469,2016,BURGER KING TILLERTORGET,1.1.1.0,Hamburger restaurants,16017414,NaN,63.358068,10.374832,BURGER KING,Tillertorget,4.528


In [47]:
#convert y values to logarithmic scale
train_data['revenue'] = np.log1p(train_data['revenue'])
train_data.head()
    

,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name,revenue
0,983540538-974187930-44774,2016,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,Hamburger restaurants,6020303,BRAGERNES TORG 13,59.743104,10.204928,MCDONALDS,Magasinet Drammen,2.944334
1,987074191-973117734-44755,2016,MCDONALD'S KLINGENBERGGATA,1.1.1.0,Hamburger restaurants,3010306,NaN,59.913759,10.734031,MCDONALDS,NaN,3.211972
2,984890265-981157303-64491,2016,BURGER KING HØNEFOSS,1.1.1.0,Hamburger restaurants,6050102,KONG RINGS GATE 1,60.164751,10.254656,BURGER KING,Kuben Hønefoss,2.839020
3,914057442-992924179-126912,2016,BURGER KING GLASSHUSPASSASJEN,1.1.1.0,Hamburger restaurants,18040102,STORGATA 12,67.283669,14.379796,BURGER KING,Glasshuspassasjen,2.331755
4,913018583-913063538-668469,2016,BURGER KING TILLERTORGET,1.1.1.0,Hamburger restaurants,16017414,NaN,63.358068,10.374832,BURGER KING,Tillertorget,1.709826


In [48]:
""" #convert them back to normal scale
train_data['revenue'] = np.exp(train_data['revenue']) -1
train_data.head() """

" #convert them back to normal scale\ntrain_data['revenue'] = np.exp(train_data['revenue']) -1\ntrain_data.head() "

In [49]:
# Replace NaN in mall_name and chain_name columns with 'No mall' and 'No chain'
train_data.mall_name = train_data.mall_name.fillna('No mall')
train_data.chain_name = train_data.chain_name.fillna('No chain')

# Dummy variable for mall or no mall
train_data.loc[train_data['mall_name'].str.contains("No mall", na=False),'mall_dummy'] = 0
train_data.loc[~(train_data['mall_name'].str.contains("No mall", na=False)),'mall_dummy'] = 1
train_data.drop(['mall_name'],axis=1, inplace=True)

# 'store_name', 'year', 'sales_channel_name', 'address' columns are redundant, remove them
train_data = train_data.drop('store_name',axis=1)
train_data = train_data.drop('year',axis=1)
train_data = train_data.drop('sales_channel_name',axis=1)
train_data = train_data.drop('address',axis=1)


In [50]:
train_data = pd.merge(train_data, grunnkrets_stripped[['grunnkrets_id', 'municipality_name']], on='grunnkrets_id', how='left')
# we get a bunch of duplicates of store_ids...? Remove them.
train_data = train_data.drop_duplicates(subset=['store_id'], keep='first')

In [51]:
chains = train_data.filter(['chain_name','revenue'], axis=1)
chains = chains.groupby('chain_name').mean()
chains = chains.rename(columns={'revenue':'mean_revenue_for_chain'})
train_data.merge(chains, how="left", on=["chain_name"])


,store_id,plaace_hierarchy_id,grunnkrets_id,lat,lon,chain_name,revenue,mall_dummy,municipality_name,mean_revenue_for_chain
0,983540538-974187930-44774,1.1.1.0,6020303,59.743104,10.204928,MCDONALDS,2.944334,1.0,Drammen,3.188035
1,987074191-973117734-44755,1.1.1.0,3010306,59.913759,10.734031,MCDONALDS,3.211972,0.0,Oslo,3.188035
2,984890265-981157303-64491,1.1.1.0,6050102,60.164751,10.254656,BURGER KING,2.839020,1.0,Ringerike,1.830112
3,914057442-992924179-126912,1.1.1.0,18040102,67.283669,14.379796,BURGER KING,2.331755,1.0,Bodø,1.830112
4,913018583-913063538-668469,1.1.1.0,16017414,63.358068,10.374832,BURGER KING,1.709826,1.0,Trondheim,1.830112
...,...,...,...,...,...,...,...,...,...,...
12854,915789943-915806929-781991,2.8.11.2,7010705,59.416276,10.480970,No chain,0.084341,0.0,Horten,1.263696
12855,917921733-917982368-868081,2.8.11.2,5020406,60.799991,10.693635,GULATING GRUPPEN,1.035317,1.0,Gjøvik,2.207821
12856,911721961-911764474-496764,2.8.11.2,2310803,59.946562,11.007659,GULATING GRUPPEN,3.669314,1.0,Skedsmo,2.207821
12857,914337046-914343372-721294,2.8.11.2,11020113,58.850261,5.735674,No chain,1.535145,1.0,Sandnes,1.263696


In [52]:
train_data = pd.merge(train_data, plaace_hierarchy[['plaace_hierarchy_id', 'lv1', 'lv2', 'lv3']], on='plaace_hierarchy_id', how='outer')
train_data['lv1']= train_data['lv1'].astype('category')
train_data['lv2']= train_data['lv2'].astype('category')
train_data['lv3']= train_data['lv3'].astype('category')

In [53]:
grunnkrets_age = grunnkrets_age.drop_duplicates(subset=['grunnkrets_id'], keep='last') # if there is value for 2016 we keep it, otherwise 2015
grunnkrets_age = grunnkrets_age.fillna(0)
grunnkrets_age = grunnkrets_age.drop('year',axis=1)
grunnkrets_age['grunnkrets_id'] = grunnkrets_age['grunnkrets_id'].astype(str)
grunnkrets_age['total_nbr_people'] = grunnkrets_age.sum(axis=1) # total number of inhabitants
""" grunnkrets_age['group1'] = grunnkrets_age.iloc[:,1:11].sum(axis=1) # 0-9 years old
grunnkrets_age['group2'] = grunnkrets_age.iloc[:,11:21].sum(axis=1) # 10-19 years old etc
grunnkrets_age['group3'] = grunnkrets_age.iloc[:,21:31].sum(axis=1)
grunnkrets_age['group4'] = grunnkrets_age.iloc[:,31:41].sum(axis=1)
grunnkrets_age['group5'] = grunnkrets_age.iloc[:,41:51].sum(axis=1)
grunnkrets_age['group6'] = grunnkrets_age.iloc[:,51:61].sum(axis=1)
grunnkrets_age['group7'] = grunnkrets_age.iloc[:,61:71].sum(axis=1)
grunnkrets_age['group8'] = grunnkrets_age.iloc[:,71:81].sum(axis=1)
grunnkrets_age['group9'] = grunnkrets_age.iloc[:,81:92].sum(axis=1) # 80-90 years old """
grunnkrets_age['grunnkrets_id'] = grunnkrets_age['grunnkrets_id'].astype(int)
#train_data = pd.merge(train_data, grunnkrets_age[['grunnkrets_id', 'total_nbr_people', 'group1', 'group2', 'group3', 'group4', 'group5', 'group6', 'group7', 'group8', 'group9']], on='grunnkrets_id', how='left')
train_data = pd.merge(train_data, grunnkrets_age[['grunnkrets_id', 'total_nbr_people']], on='grunnkrets_id', how='left')

/var/folders/v4/l760qmd51vl3yg0k0ly6n5fw0000gn/T/ipykernel_45070/4143716196.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  grunnkrets_age['total_nbr_people'] = grunnkrets_age.sum(axis=1) # total number of inhabitants


In [54]:
# Number of people per store in each grunnkrets regardless of hierarchy
number_stores = train_data['grunnkrets_id'].value_counts().rename_axis('grunnkrets_id').reset_index(name='store_counts_total') # Not including NaN (stores without a grunnkrets_id)
grunnkrets_stripped = pd.merge(grunnkrets_stripped, number_stores[['grunnkrets_id', 'store_counts_total']], on='grunnkrets_id', how='left')
grunnkrets_stripped.store_counts_total = grunnkrets_stripped.store_counts_total.fillna(0)
grunnkrets_stripped = pd.merge(grunnkrets_stripped, grunnkrets_age[['grunnkrets_id', 'total_nbr_people']], on='grunnkrets_id', how='left')
grunnkrets_stripped['nbr_people_per_store_in_grunnkrets'] = grunnkrets_stripped['total_nbr_people']/grunnkrets_stripped['store_counts_total']
train_data = pd.merge(train_data, grunnkrets_stripped[['grunnkrets_id', 'nbr_people_per_store_in_grunnkrets']], on='grunnkrets_id', how='left')


In [55]:
# Number of people per store in each grunnkrets in lv2
counts = train_data[["store_id", "grunnkrets_id", "lv2"]].groupby(
    ["grunnkrets_id", "lv2"]
).count().reset_index()
counts.columns = ["grunnkrets_id", "lv2", "counts_gr_lv2"]
train_data = train_data.merge(counts, how="left", on=["grunnkrets_id", "lv2"])

In [56]:
#train_data.update(train_data[['total_nbr_people','group1','group2','group3','group4','group5','group6','group7','group8','group9']].fillna(0))
#train_data.update(train_data[['nbr_people_per_store_in_grunnkrets','nbr_people_per_km2']].fillna(0))


In [57]:
# Number of stores in same lv2 in each municipality
nbr_in_municipality = train_data[["store_id","municipality_name", "lv2"]].groupby(
    ["municipality_name", "lv2"]
).count().reset_index()
nbr_in_municipality.columns = ["municipality_name", "lv2", "counts_municipality_lv2"]
train_data = train_data.merge(nbr_in_municipality, how="left", on=["municipality_name", "lv2"])

In [58]:
# Mean revenue for each lv1 in each municipality
municipalities = train_data[["municipality_name", "lv1", "revenue"]].groupby(
    ["municipality_name", "lv1"]
).mean().reset_index()
municipalities = municipalities.rename(columns={'revenue':'mean_revenue_for_municipality_and_level1'})
train_data = train_data.merge(municipalities, how="left", on=["municipality_name", "lv1"])
# we get a bunch of duplicates of store_ids...? Remove them.
train_data = train_data.drop_duplicates(subset=['store_id'], keep='first')
train_data.head()

,store_id,plaace_hierarchy_id,grunnkrets_id,lat,lon,chain_name,revenue,mall_dummy,municipality_name,lv1,lv2,lv3,total_nbr_people,nbr_people_per_store_in_grunnkrets,counts_gr_lv2,counts_municipality_lv2,mean_revenue_for_municipality_and_level1
0,983540538-974187930-44774,1.1.1.0,6020303.0,59.743104,10.204928,MCDONALDS,2.944334,1.0,Drammen,1,1.1,1.1.1,157.0,3.140000,28.0,92.0,1.685027
2,987074191-973117734-44755,1.1.1.0,3010306.0,59.913759,10.734031,MCDONALDS,3.211972,0.0,Oslo,1,1.1,1.1.1,NaN,NaN,56.0,924.0,1.684780
4,984890265-981157303-64491,1.1.1.0,6050102.0,60.164751,10.254656,BURGER KING,2.839020,1.0,Ringerike,1,1.1,1.1.1,372.0,26.571429,8.0,34.0,1.600517
6,914057442-992924179-126912,1.1.1.0,18040102.0,67.283669,14.379796,BURGER KING,2.331755,1.0,Bodø,1,1.1,1.1.1,514.0,10.936170,16.0,62.0,1.738266
8,913018583-913063538-668469,1.1.1.0,16017414.0,63.358068,10.374832,BURGER KING,1.709826,1.0,Trondheim,1,1.1,1.1.1,NaN,NaN,6.0,196.0,1.805180


In [59]:
# Mean revenue for each lv2 in each municipality
municipalities = train_data[["municipality_name", "lv2", "revenue"]].groupby(
    ["municipality_name", "lv2"]
).mean().reset_index()
municipalities = municipalities.rename(columns={'revenue':'mean_revenue_for_municipality_and_level2'})
train_data = train_data.merge(municipalities, how="left", on=["municipality_name", "lv2"])
# we get a bunch of duplicates of store_ids...? Remove them.
train_data = train_data.drop_duplicates(subset=['store_id'], keep='first')
train_data.head()


,store_id,plaace_hierarchy_id,grunnkrets_id,lat,lon,chain_name,revenue,mall_dummy,municipality_name,lv1,lv2,lv3,total_nbr_people,nbr_people_per_store_in_grunnkrets,counts_gr_lv2,counts_municipality_lv2,mean_revenue_for_municipality_and_level1,mean_revenue_for_municipality_and_level2
0,983540538-974187930-44774,1.1.1.0,6020303.0,59.743104,10.204928,MCDONALDS,2.944334,1.0,Drammen,1,1.1,1.1.1,157.0,3.140000,28.0,92.0,1.685027,1.657954
1,987074191-973117734-44755,1.1.1.0,3010306.0,59.913759,10.734031,MCDONALDS,3.211972,0.0,Oslo,1,1.1,1.1.1,NaN,NaN,56.0,924.0,1.684780,1.693765
2,984890265-981157303-64491,1.1.1.0,6050102.0,60.164751,10.254656,BURGER KING,2.839020,1.0,Ringerike,1,1.1,1.1.1,372.0,26.571429,8.0,34.0,1.600517,1.640187
3,914057442-992924179-126912,1.1.1.0,18040102.0,67.283669,14.379796,BURGER KING,2.331755,1.0,Bodø,1,1.1,1.1.1,514.0,10.936170,16.0,62.0,1.738266,1.703086
4,913018583-913063538-668469,1.1.1.0,16017414.0,63.358068,10.374832,BURGER KING,1.709826,1.0,Trondheim,1,1.1,1.1.1,NaN,NaN,6.0,196.0,1.805180,1.792743


In [60]:
# Mean revenue for each lv3 in each municipality
municipalities = train_data[["municipality_name", "lv3", "revenue"]].groupby(
    ["municipality_name", "lv3"]
).mean().reset_index()
municipalities = municipalities.rename(columns={'revenue':'mean_revenue_for_municipality_and_level3'})
train_data = train_data.merge(municipalities, how="left", on=["municipality_name", "lv3"])
# we get a bunch of duplicates of store_ids...? Remove them.
train_data = train_data.drop_duplicates(subset=['store_id'], keep='first')
train_data.head()

,store_id,plaace_hierarchy_id,grunnkrets_id,lat,lon,chain_name,revenue,mall_dummy,municipality_name,lv1,lv2,lv3,total_nbr_people,nbr_people_per_store_in_grunnkrets,counts_gr_lv2,counts_municipality_lv2,mean_revenue_for_municipality_and_level1,mean_revenue_for_municipality_and_level2,mean_revenue_for_municipality_and_level3
0,983540538-974187930-44774,1.1.1.0,6020303.0,59.743104,10.204928,MCDONALDS,2.944334,1.0,Drammen,1,1.1,1.1.1,157.0,3.140000,28.0,92.0,1.685027,1.657954,2.944334
1,987074191-973117734-44755,1.1.1.0,3010306.0,59.913759,10.734031,MCDONALDS,3.211972,0.0,Oslo,1,1.1,1.1.1,NaN,NaN,56.0,924.0,1.684780,1.693765,2.083126
2,984890265-981157303-64491,1.1.1.0,6050102.0,60.164751,10.254656,BURGER KING,2.839020,1.0,Ringerike,1,1.1,1.1.1,372.0,26.571429,8.0,34.0,1.600517,1.640187,2.839020
3,914057442-992924179-126912,1.1.1.0,18040102.0,67.283669,14.379796,BURGER KING,2.331755,1.0,Bodø,1,1.1,1.1.1,514.0,10.936170,16.0,62.0,1.738266,1.703086,2.468111
4,913018583-913063538-668469,1.1.1.0,16017414.0,63.358068,10.374832,BURGER KING,1.709826,1.0,Trondheim,1,1.1,1.1.1,NaN,NaN,6.0,196.0,1.805180,1.792743,2.427614


In [61]:
col_at_end = ['revenue']
train_data = train_data[[c for c in train_data if c not in col_at_end] + [c for c in col_at_end if c in train_data]]

In [62]:
train_data.head()

,store_id,plaace_hierarchy_id,grunnkrets_id,lat,lon,chain_name,mall_dummy,municipality_name,lv1,lv2,lv3,total_nbr_people,nbr_people_per_store_in_grunnkrets,counts_gr_lv2,counts_municipality_lv2,mean_revenue_for_municipality_and_level1,mean_revenue_for_municipality_and_level2,mean_revenue_for_municipality_and_level3,revenue
0,983540538-974187930-44774,1.1.1.0,6020303.0,59.743104,10.204928,MCDONALDS,1.0,Drammen,1,1.1,1.1.1,157.0,3.140000,28.0,92.0,1.685027,1.657954,2.944334,2.944334
1,987074191-973117734-44755,1.1.1.0,3010306.0,59.913759,10.734031,MCDONALDS,0.0,Oslo,1,1.1,1.1.1,NaN,NaN,56.0,924.0,1.684780,1.693765,2.083126,3.211972
2,984890265-981157303-64491,1.1.1.0,6050102.0,60.164751,10.254656,BURGER KING,1.0,Ringerike,1,1.1,1.1.1,372.0,26.571429,8.0,34.0,1.600517,1.640187,2.839020,2.839020
3,914057442-992924179-126912,1.1.1.0,18040102.0,67.283669,14.379796,BURGER KING,1.0,Bodø,1,1.1,1.1.1,514.0,10.936170,16.0,62.0,1.738266,1.703086,2.468111,2.331755
4,913018583-913063538-668469,1.1.1.0,16017414.0,63.358068,10.374832,BURGER KING,1.0,Trondheim,1,1.1,1.1.1,NaN,NaN,6.0,196.0,1.805180,1.792743,2.427614,1.709826


In [63]:
print(list(train_data.columns))

['store_id', 'plaace_hierarchy_id', 'grunnkrets_id', 'lat', 'lon', 'chain_name', 'mall_dummy', 'municipality_name', 'lv1', 'lv2', 'lv3', 'total_nbr_people', 'nbr_people_per_store_in_grunnkrets', 'counts_gr_lv2', 'counts_municipality_lv2', 'mean_revenue_for_municipality_and_level1', 'mean_revenue_for_municipality_and_level2', 'mean_revenue_for_municipality_and_level3', 'revenue']


In [64]:
#remove all the features we dont use for our model
#train_data = train_data.drop('store_id',axis=1)
train_data = train_data.drop('plaace_hierarchy_id',axis=1)
#train_data = train_data.drop('grunnkrets_id',axis=1)
train_data = train_data.drop('chain_name',axis=1)
train_data = train_data.drop('municipality_name',axis=1)
train_data = train_data.drop('lv3',axis=1)

In [65]:
train_data.to_csv('../data/modified_data.csv')
train_data.head()


,store_id,grunnkrets_id,lat,lon,mall_dummy,lv1,lv2,total_nbr_people,nbr_people_per_store_in_grunnkrets,counts_gr_lv2,counts_municipality_lv2,mean_revenue_for_municipality_and_level1,mean_revenue_for_municipality_and_level2,mean_revenue_for_municipality_and_level3,revenue
0,983540538-974187930-44774,6020303.0,59.743104,10.204928,1.0,1,1.1,157.0,3.140000,28.0,92.0,1.685027,1.657954,2.944334,2.944334
1,987074191-973117734-44755,3010306.0,59.913759,10.734031,0.0,1,1.1,NaN,NaN,56.0,924.0,1.684780,1.693765,2.083126,3.211972
2,984890265-981157303-64491,6050102.0,60.164751,10.254656,1.0,1,1.1,372.0,26.571429,8.0,34.0,1.600517,1.640187,2.839020,2.839020
3,914057442-992924179-126912,18040102.0,67.283669,14.379796,1.0,1,1.1,514.0,10.936170,16.0,62.0,1.738266,1.703086,2.468111,2.331755
4,913018583-913063538-668469,16017414.0,63.358068,10.374832,1.0,1,1.1,NaN,NaN,6.0,196.0,1.805180,1.792743,2.427614,1.709826
